In [69]:
import numpy as np
from netCDF4 import Dataset
import cartopy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import torch
import torchvision
import sys
import os

from sklearn.decomposition import PCA

# Load netcdf file
data_dir = '/home/jcurtis2/hackathon_data/'
wrf_filename = '%straining.nc' % data_dir
ncf = Dataset(wrf_filename, "r", format="NETCDF4")

# WRF map projection
cart_proj = cartopy.crs.LambertConformal(central_longitude=-97,central_latitude=38)
# Data projection
data_proj = cartopy.crs.PlateCarree()

# Static data
lats = ncf.variables['XLAT'][0,:,:]
lons = ncf.variables['XLONG'][0,:,:]

i_time = -1 # Scecify output time to plot
i_z = 0 # Specify level to plot


data = ncf.variables['TOT_NUM_CONC'][:]

# (nt,nz,ny,nx) = data.shape
# print(nt,nz,ny,nx)


# x_s = 1
# x_e = nx-1
# y_s = 1
# y_e = ny-1
# z_s = 0
# z_e = nz
# t_s = 0
# t_e = nt
# print(data[t_s:t_e,z_s:z_e,y_s:y_e,x_s:x_e].shape)

# data[t_s:t_e,z_s:z_e,y_s:y_e,x_s:x_e]

# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)
 





AttributeError: 'PCA' object has no attribute 'explained_variance_ratio_'

In [70]:
# import sys
# import os
# import torch

aero = open('/home/mingzex2/hak_data/x_aerosols.txt', 'r')
gas = open('/home/mingzex2/hak_data/x_gases.txt', 'r')
aero_e = aero.readlines()
gas_e = gas.readlines()

elem = []

for i in aero_e:
    elem.append(i.strip())
    
for i in gas_e:
    elem.append(i.strip())

print(elem)





['TOT_NUM_CONC', 'TOT_MASS_CONC', 'pmc_SO4', 'pmc_NO3', 'pmc_Cl', 'pmc_NH4', 'pmc_ARO1', 'pmc_ARO2', 'pmc_ALK1', 'pmc_OLE1', 'pmc_API1', 'pmc_API2', 'pmc_LIM1', 'pmc_LIM2', 'pmc_OC', 'pmc_BC', 'pmc_H2O', 'h2so4', 'hno3', 'hcl', 'nh3', 'no', 'no2', 'no3', 'n2o5', 'hono', 'hno4', 'o3', 'o1d', 'O3P', 'oh', 'ho2', 'h2o2', 'co', 'so2', 'ch4', 'c2h6', 'ch3o2', 'ethp', 'hcho', 'ch3oh', 'ANOL', 'ch3ooh', 'ETHOOH', 'ald2', 'hcooh', 'RCOOH', 'c2o3', 'pan', 'aro1', 'aro2', 'alk1', 'ole1', 'api1', 'api2', 'lim1', 'lim2', 'par', 'AONE', 'mgly', 'eth', 'OLET', 'OLEI', 'tol', 'xyl', 'cres', 'to2', 'cro', 'open', 'onit', 'rooh', 'ro2', 'ano2', 'nap', 'xo2', 'xpar', 'isop', 'isoprd', 'isopp', 'isopn', 'isopo2', 'api', 'lim', 'dms', 'msa', 'dmso', 'dmso2', 'ch3so2h', 'ch3sch2oo', 'ch3so2', 'ch3so3', 'ch3so2oo', 'ch3so2ch2oo', 'SULFHOX']


In [71]:
for variable in elem:
#     data = ncf.variables[variable]
#     data = torch.from_numpy(np.array(data))
#     print(type(data))
    print(variable, ncf.variables[variable].shape)

TOT_NUM_CONC (133, 39, 159, 169)
TOT_MASS_CONC (133, 39, 159, 169)
pmc_SO4 (133, 39, 159, 169)
pmc_NO3 (133, 39, 159, 169)
pmc_Cl (133, 39, 159, 169)
pmc_NH4 (133, 39, 159, 169)
pmc_ARO1 (133, 39, 159, 169)
pmc_ARO2 (133, 39, 159, 169)
pmc_ALK1 (133, 39, 159, 169)
pmc_OLE1 (133, 39, 159, 169)
pmc_API1 (133, 39, 159, 169)
pmc_API2 (133, 39, 159, 169)
pmc_LIM1 (133, 39, 159, 169)
pmc_LIM2 (133, 39, 159, 169)
pmc_OC (133, 39, 159, 169)
pmc_BC (133, 39, 159, 169)
pmc_H2O (133, 39, 159, 169)
h2so4 (133, 39, 159, 169)
hno3 (133, 39, 159, 169)
hcl (133, 39, 159, 169)
nh3 (133, 39, 159, 169)
no (133, 39, 159, 169)
no2 (133, 39, 159, 169)
no3 (133, 39, 159, 169)
n2o5 (133, 39, 159, 169)
hono (133, 39, 159, 169)
hno4 (133, 39, 159, 169)
o3 (133, 39, 159, 169)
o1d (133, 39, 159, 169)
O3P (133, 39, 159, 169)
oh (133, 39, 159, 169)
ho2 (133, 39, 159, 169)
h2o2 (133, 39, 159, 169)
co (133, 39, 159, 169)
so2 (133, 39, 159, 169)
ch4 (133, 39, 159, 169)
c2h6 (133, 39, 159, 169)
ch3o2 (133, 39, 159, 169

In [77]:
num_of_features = 10
i = 0

transform = torchvision.transforms.Resize((157, 157))

torchData = torch.empty(num_of_features, 133, 39, 157, 157)
for variable in elem:
    data = ncf.variables[variable]
    data = torch.from_numpy(np.array(data))
    data = data[:,:,1:-1,1:-1]
    data=transform(data)
    torchData[i] = data
    i += 1
    print(type(data))
    print(data.shape)
    if i == num_of_features:
        break


#     print(variable, ncf.variables[variable].shape)

<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])
<class 'torch.Tensor'>
torch.Size([133, 39, 157, 157])


In [78]:
data = torchData
print(data.size())

torch.Size([10, 133, 39, 157, 157])


In [79]:
data = torch.permute(data, (1, 0, 2,3,4))

In [80]:
print(data.size())

torch.Size([133, 10, 39, 157, 157])
